In [2]:
from langchain_core.messages import AIMessage

from app.core.config import settings
from app.database.milvus import get_milvus_client
get_milvus_client()

2024-12-25 20:28:37,468 - fastapi_project - INFO - Connected to Milvus at https://in01-8b96d805fab7a0f.aws-ap-northeast-1.vectordb.zillizcloud.com:19533


In [7]:
from app.features.manual.models.model import create_openai_model
from app.features.manual.models.schemas import AnswerWithHistory
llm = create_openai_model('gpt-4o-mini')
structured_llm_grader = llm.with_structured_output(AnswerWithHistory)

In [8]:
from langchain_core.prompts import PromptTemplate
history_base_prompt = PromptTemplate.from_template(
        """
        다음은 이전 대화 내용만을 참고하여 응답을 생성하기 위한 프로세스입니다. 외부 지식이나 학습된 데이터를 사용하지 않고 오직 제공된 대화 내용을 기반으로 응답합니다.

        # 응답 가능성 판단 및 생성

        ## 절차
        1. **이전 대화 분석**: 이전 대화의 내용을 검토하여 대화의 주제 및 구체적으로 제기된 질문을 파악합니다.
        2. **질문 또는 프롬프트 식별**: 명시적 또는 암시적인 질문이나 요청을 식별합니다.
        3. **응답 가능성 평가**: 제공된 이전 대화 내용만으로 충분한 정보가 있는지 판단하고, 응답 가능 여부를 결정합니다.
           - 응답 가능하면 `'yes'`를 출력합니다.
           - 응답이 불가능하면 `'no'`를 출력합니다.
        4. **응답 생성**: 
           - 가능(`'yes'`)하면 이전 대화 내용만을 바탕으로 논리적이고 맥락에 맞는 응답을 작성합니다.
           - 불가능(`'no'`)하면 추가 정보가 필요하다는 점을 명시합니다.

        ## 제한 사항
        - 외부 지식, 학습된 데이터, 일반 상식 등을 사용하지 않습니다.
        - 이전 대화 내용 외의 정보를 가정하거나 추가하지 않습니다.

        ## 출력 형식
        - 출력은 **마크다운(Markdown)** 형식으로 작성합니다.
        - 응답 가능 여부(`yes` 또는 `no`)를 표시하고, 이에 따른 적절한 내용을 제공합니다.

        ## 이전 대화 내용: {chat_history}

        ## 질문: {question}
        """)

contextual_responder = history_base_prompt | structured_llm_grader

In [9]:
from langchain.schema import AIMessage, HumanMessage
chat_list = [
    HumanMessage(content="제네시스 블루투스 시스템에 내 휴대폰을 어떻게 연결하나요?"),
    AIMessage(content="제네시스 블루투스 시스템에 휴대폰을 연결하려면, 인포테인먼트 화면에서 '설정' 메뉴로 이동한 후 '기기 연결'을 선택하고 '블루투스'를 누르세요. 휴대폰에서 블루투스를 활성화하고, 사용 가능한 장치 목록에서 차량을 선택하세요."),

    HumanMessage(content="타이어 압력 경고등이 켜지면 어떻게 해야 하나요?"),
    AIMessage(content="타이어 압력 경고등이 켜지면, 타이어 압력 모니터링 시스템이나 타이어 게이지를 사용하여 압력을 확인하세요. 권장 압력은 운전석 문틀의 스티커에서 확인할 수 있으며, 부족한 타이어는 적정 압력으로 채우세요."),

    HumanMessage(content="운전석 좌석을 편안하게 조정하려면 어떻게 해야 하나요?"),
    AIMessage(content="운전석 좌석을 조정하려면, 좌석 옆에 있는 전동 좌석 조정 버튼을 사용하세요. 좌석을 앞뒤로 이동하거나 높낮이를 조정하고, 등받이 각도와 요추 지지대를 조절할 수 있습니다."),

    HumanMessage(content="제네시스 G80의 비상용 키트는 어디에 있나요?"),
    AIMessage(content="제네시스 G80의 비상용 키트는 일반적으로 트렁크에 있으며, 바닥판 아래 수납공간이나 옆면 패널에 위치할 수 있습니다. 정확한 위치는 차량 매뉴얼을 참조하세요."),

    HumanMessage(content="크루즈 컨트롤 기능은 어떻게 활성화하나요?"),
    AIMessage(content="크루즈 컨트롤 기능을 활성화하려면, 스티어링 휠에 있는 'CRUISE' 버튼을 누르세요. 원하는 속도로 가속한 후 'SET' 버튼을 눌러 시스템을 작동시킬 수 있습니다."),

    HumanMessage(content="제네시스가 시동이 걸리지 않으면 어떻게 해야 하나요?"),
    AIMessage(content="제네시스의 시동이 걸리지 않을 경우, 우선 스마트 키의 배터리가 충분한지 확인하세요. 배터리가 부족하다면, 키를 시동 버튼 근처에 가까이 대고 시도하세요. 또한 차량 배터리가 충전되었는지 확인하고, 문제가 지속되면 제네시스 딜러에 문의하세요."),

    HumanMessage(content="주차할 때 후방 카메라를 어떻게 활성화하나요?"),
    AIMessage(content="주차할 때 후방 카메라를 활성화하려면, 기어를 후진으로 변경하세요. 그러면 인포테인먼트 화면에 후방 카메라 화면이 자동으로 표시됩니다."),

    HumanMessage(content="제네시스 GV70에 적합한 연료는 무엇인가요?"),
    AIMessage(content="제네시스 GV70에 권장되는 연료는 옥탄가 91 이상의 고급 무연 휘발유입니다. 자세한 내용은 차량 매뉴얼을 참조하세요."),

    HumanMessage(content="내비게이션 시스템을 어떻게 업데이트하나요?"),
    AIMessage(content="내비게이션 시스템을 업데이트하려면, 제네시스 내비게이션 업데이트 웹사이트를 방문하여 최신 소프트웨어를 USB 드라이브에 다운로드하세요. USB를 차량의 USB 포트에 연결한 후 화면의 지침에 따라 업데이트를 완료하세요."),

    HumanMessage(content="겨울철에 열선 시트를 어떻게 작동시키나요?"),
    AIMessage(content="열선 시트를 작동시키려면, 공조 제어 패널에 있는 열선 시트 버튼을 누르세요. 동일한 버튼을 사용하여 열선 강도를 조절할 수 있습니다.")
]


In [14]:

question = '열선 시트 사용법?'
response = contextual_responder.invoke({"chat_history": chat_list, "question": question})
print(response)

binary_score='yes' answer='열선 시트를 작동시키려면, 공조 제어 패널에 있는 열선 시트 버튼을 누르세요. 동일한 버튼을 사용하여 열선 강도를 조절할 수 있습니다.'


# 판단 노드

In [1]:
from app.core.config import settings

2024-12-25 23:07:51,230 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_URI
2024-12-25 23:07:51,429 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_USER
2024-12-25 23:07:51,607 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_PASSWORD
2024-12-25 23:07:51,796 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_DB_NAME
2024-12-25 23:07:51,992 - fastapi_project - INFO - Retrieved parameter: /tailorlink/openai/OPENAI_API_KEY
2024-12-25 23:07:52,182 - fastapi_project - INFO - Retrieved parameter: /tailorlink/milvus/MILVUS_URI
2024-12-25 23:07:52,381 - fastapi_project - INFO - Retrieved parameter: /tailorlink/milvus/MILVUS_TOKEN
2024-12-25 23:07:52,553 - fastapi_project - INFO - Retrieved parameter: /tailorlink/langchain/LANGCHAIN_API_KEY
2024-12-25 23:07:52,726 - fastapi_project - INFO - Retrieved parameter: /tailorlink/langchain/LANGCHAIN_ENDPOINT


In [2]:
from app.features.manual.models.prompt_templates import create_check_genesis_and_split_prompt
from app.features.manual.models.model import create_openai_model
from app.features.manual.models.schemas import QuestionList

llm = create_openai_model()
structured_llm_grader = llm.with_structured_output(QuestionList)
prompt = create_check_genesis_and_split_prompt()

chain = prompt | structured_llm_grader

In [3]:
prompt.input_variables

['car_model', 'question']

In [6]:
from app.features.manual.tools.tools import get_genesis_model
car_list = get_genesis_model()

2024-12-25 23:08:02,171 - fastapi_project - INFO - Connected to MySQL database: mydatabase on my-database.cbuom6aeon9v.ap-northeast-2.rds.amazonaws.com


In [13]:
question = 'g70 shooting 현재 가격?'
chain.invoke({"car_model": car_list, "question": question})

QuestionList(vailid_question=True, reason='질문이 제네시스 차량(g70 shooting)과 관련되어 있으며, 단일 질문으로 판단됨.', print="['g70 shooting 현재 가격?']", question_list=['g70 shooting 현재 가격?'])

# 전체 파이프 라인 테스트